In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from dateutil.relativedelta import relativedelta
import clipboard
import datetime
import os
import pickle
import pyperclip
import time

In [3]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
LOGIN_URL = 'https://x.com/login'

# Chromeオプションの設定
chrome_options = Options()
chrome_options.add_argument('--disable-popup-blocking')  # ポップアップブロック
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--user-agent=%s' % USER_AGENT)
chrome_options.add_argument('--window-size=1920,1080')

In [8]:
def login():
    global LOGIN_URL

    username = input('usernameを入力してください:')
    password = input('passwordを入力してください:')

    driver = webdriver.Chrome(options=chrome_options)

    #xのログイン画面にアクセス
    driver.get(LOGIN_URL)

    #名前入力画面が出現するまでロードを待つ
    print('入力画面待機中です')
    try:
        WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.TAG_NAME, 'input'))
        ).send_keys(f'{username}')
        driver.find_element(By.XPATH, '//button[2]').click()
    except Exception:
        print('入力画面をロード中にエラーが発生しました')
        driver.save_screenshot('username-input-connection-errors.png')

    #メールアドレス確認画面が出現するまでロードを待つ
    print('メールアドレス確認画面待機中です')
    try:
        mailadress_input = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.TAG_NAME, 'input'))
        )
        mailadress = input('メールアドレスを入力してください：')
        mailadress_input.send_keys(mailadress)
        mailadress_input.send_keys(Keys.ENTER)
    except Exception:
        print('パスワード画面に遷移します')
        pass

    #パスワード入力画面が出現するまでロードを待つ
    print('パスワード画面待機中です')
    try:
        password_input = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.NAME, 'password'))
        )
        password_input.send_keys(f'{password}')
        password_input.send_keys(Keys.ENTER)
    except Exception:
        print('パスワード画面をロード中にエラーが発生しました')
        driver.save_screenshot('password-connection-errors.png')

    #cookieを取得して保存する
    time.sleep(10)
    cookies = driver.get_cookies()

    with open('cookies', 'wb') as f:
        pickle.dump(cookies, f)

    print('ログインしました')
    driver.quit()

In [100]:
def connect_X(driver, url):
    global cookies

    #xのアカウント画面にアクセス
    driver.get(url)

    #cookieを設定して再度アクセス
    for c in cookies:
        driver.add_cookie(c)
    driver.get(url)

    #投稿内容が出現するまでロードを待つ
    print('待機中です')
    try:
        wait = WebDriverWait(driver, 30)
        wait.until(EC.presence_of_all_elements_located)
    except Exception:
        print('ロード中にエラーが発生しました')
        driver.save_screenshot('connection-errors.png')
    
    #body要素を取得
    body = driver.find_element(By.ID, 'react-root')

    return body

In [79]:
def get_tweet_id(driver, dom):
    driver.set_permissions('clipboard-write', 'granted')
    driver.set_permissions('clipboard-read', 'granted')

    #リンクをコピーする
    dom.find_elements(By.TAG_NAME, 'button')[-1].click()
    time.sleep(0.1)
    driver.find_elements(By.CSS_SELECTOR, "[role='menuitem']")[0].click()
    time.sleep(1)S
    
    #コピーしたリンクを返す
    link = pyperclip.paste()
    return link

In [99]:
def fetch_posts(driver, url, i: int = 1):
    global target_date

    body = connect_X(driver, url)
    
    post_list = []

    #スクロールさせる幅(初めは0)
    y = 0

    #今日の日付からdateで渡された日数分の投稿を取得
    while True:
        print(f'index:{i} の投稿を取得します')

        #投稿の中に埋め込まれているtimeタグを取得
        try:
            WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, f'//section/div/div/div[{i}]'))
                )
            post_dom = body.find_element(By.XPATH, f'//section/div/div/div[{i}]')
            post_text = post_dom.text.splitlines()
        
        #取得できない場合はスクロールを行なってdom上に表示させる
        except Exception:
            print(f'index:{i} の投稿はdom上に存在しません')

            #スクロール先のy座標(1投稿が縦530px)
            y += 530
            print(y)
            # スクロールして投稿を読み込むまで待機
            driver.execute_script(f'window.scrollTo(0, {y});') 
            
            try:
                print('投稿を読み込んでいます')
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, f'//section/div/div/div[{i}]'))
                )
            except NoSuchElementException:
                driver.save_screenshot('NoSuchContent.png')

            continue

        #投稿の中のtimeタグを取得してtarget_dateよりも新しい日付か確認する
        try:
            time = post_dom.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
            
            s_format = '%Y-%m-%dT%H:%M:%S.000Z'
            dt_time = datetime.datetime.strptime(time, s_format)

        #取得できない場合は投稿以外の要素を取得しているためループを回す
        except:
            i += 1
            continue

        #ピン留された投稿の場合はループを回す
        if post_text[0] == 'Pinned':
            i += 1
            continue
        #新しい投稿の場合はpost_listに内容を追加する
        elif target_date < dt_time:
            get_tweet_id(driver, post_dom)

            post_list.append(post_text)
            driver.close()
            break
            # i += 1
            # continue
        #投稿は日付順になっており、target_dateよりも古い場合これ以上新しい投稿はないと判断してループから抜ける
        else:
            driver.close()
            break
            
    return post_list

In [11]:
def calculate_target_date(mode: str, number_of_date: str):
    try:
        int_number_of_date = int(number_of_date)
    except:
        #入力された日数が数字でないためエラー発生
        #再度入力をもらい再帰実行
        mode, period = input('''入力をやり直してください。\n
                             遡る期間数は数字で入力してください。:''').split()
        calculate_target_date(mode, period)

    today = datetime.datetime.today()
    
    match mode:
        #日数で遡る
        case 'd' | 'D':
            target_date = today - datetime.timedelta(days=int_number_of_date)
            return target_date
        
        #週数で遡る
        case 'w' | 'W':
            target_date = today - datetime.timedelta(weeks=int_number_of_date)
            return target_date
        
        #月数で遡る
        case 'm' | 'M':
            target_date = today - relativedelta(months=int_number_of_date)
            return target_date
        
        #いずれの文字にもマッチしない場合は再度入力を受け取る
        case _:
            mode, period = input('''入力をやり直してください。\n
                             遡る間隔は日数(D)、週数(W)、月数(M)から選び、アルファベットで入力してください:''').split()
            calculate_target_date(mode, period)

In [4]:
#cookiesファイルがなければログイン関数を実行してcookieを取得する
if os.path.isfile('cookies') != True:
    cookies = login()
#ファイルがあればcookieを読み込む
else:
    with open ('cookies', 'rb') as f:
        cookies = pickle.load(f)

In [97]:
driver = webdriver.Chrome(options=chrome_options)
driver.execute_script('const newProto = navigator.__proto__;delete newProto.webdriver;navigator.__proto__ = newProto;')

# 指定した要素が見つかるまでの待ち時間を設定する 今回は最大10秒待機する
driver.implicitly_wait(10)

#検索するアカウント
target = 'HAL_Laboratory'

url = f'https://x.com/{target}/'

In [98]:
mode, period = input('''遡る間隔と遡る期間数を入力してください。\n
                     間隔は日数(D)、週数(W)、月数(M)で遡ることができます。\n
                     入力例(日数で遡って、最新から2日前までの投稿を取得する場合):D 2''').split()    

target_date = calculate_target_date(mode, period)
post_list = fetch_posts(driver, url)

In [92]:
post_list

['株式会社ハル研究所\n@HAL_Laboratory\n·\n8h\n＼ハル研ブログ\u3000新記事掲載！／\n\n新人研修を振り返ってシリーズ第2弾！\n今回はデザイナーの新人が、技術研修とグループワーク研修の内容や、社内の雰囲気を紹介しています。ぜひご覧ください。\nハル研究所ウェブサイト｜HAL Laboratory, Inc.\nFrom hallab.co.jp\n30\n202\n14K',
 '株式会社ハル研究所 reposted\n星のカービィ\n@Kirby_JP\n·\nJul 11\nNintendo Switch Online加入者限定のスマートフォンアプリ『Nintendo Music』に、プレイリスト「夏に聴きたい」が追加されました。\nカービィシリーズからは、「リーフリゾート」や「泳げ！ひろびろビーチ」といった数曲をお楽しみいただけます。\nhttps://nintendo.com/jp/nintendo-music/index.html…\n4\n685\n2.9K\n128K',
 '株式会社ハル研究所\n@HAL_Laboratory\n·\nJul 9\n＼ハル研ブログ\u3000新記事掲載！／\n\n4月に入社した新人たちも、7月にいよいよ配属となりました。\n今日はプログラマーの新人が約3か月間の新人研修を振り返り、研修の内容やそこで学んだことをご紹介。ぜひご覧ください。\nハル研究所ウェブサイト｜HAL Laboratory, Inc.\nFrom hallab.co.jp\n68\n572\n36K',
 '株式会社ハル研究所\n@HAL_Laboratory\n·\nJul 4\n＼ハル研ブログ\u3000新記事掲載！／\n\n社内のデザイナーたちが自主的に集まって開催する「デッサン会」の様子を、運営を担当する若手デザイナーがご紹介。\n自分の中にある「既成概念」を取り払う訓練としてデッサンに取り組んでいるのだとか。ぜひご覧ください。\nハル研究所ウェブサイト｜HAL Laboratory, Inc.\nFrom hallab.co.jp\n3\n102\n564\n40K']

In [56]:
connect_X(driver, url)
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: headless chrome=116.0.5845.187)
Stacktrace:
0   chromedriver                        0x000000010899d638 chromedriver + 5002808
1   chromedriver                        0x0000000108994a53 chromedriver + 4966995
2   chromedriver                        0x0000000108545a57 chromedriver + 449111
3   chromedriver                        0x0000000108529450 chromedriver + 332880
4   chromedriver                        0x0000000108529343 chromedriver + 332611
5   chromedriver                        0x00000001085476c2 chromedriver + 456386
6   chromedriver                        0x00000001085cc9a7 chromedriver + 1001895
7   chromedriver                        0x00000001085b21c3 chromedriver + 893379
8   chromedriver                        0x000000010857ddf9 chromedriver + 679417
9   chromedriver                        0x000000010857ef

In [324]:
login()

入力画面待機中です
メールアドレス確認画面待機中です
パスワード画面に遷移します
パスワード画面待機中です


InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
0   chromedriver                        0x000000010cbf9638 chromedriver + 5002808
1   chromedriver                        0x000000010cbf0a53 chromedriver + 4966995
2   chromedriver                        0x000000010c7a1896 chromedriver + 448662
3   chromedriver                        0x000000010c7d9446 chromedriver + 676934
4   chromedriver                        0x000000010c80e2f6 chromedriver + 893686
5   chromedriver                        0x000000010c80a002 chromedriver + 876546
6   chromedriver                        0x000000010c809590 chromedriver + 873872
7   chromedriver                        0x000000010c76906a chromedriver + 217194
8   chromedriver                        0x000000010cbb62d9 chromedriver + 4727513
9   chromedriver                        0x000000010cbbb